<h1 style="text-align: center; font-size: 2.5em; padding: 30px;">Weryfikacja mojego algorytmu Algen</h1>
<h4 style="color: gray;">


Podejście:<br>
<ol>
    <li> Obliczenie współczynników za pomocą regresji liniowej z pakietu scikit-learn
    <li> Obliczenie współczynników za pomocą macierzy pseudo-inwersji Moore'a-Penrose'a
    </ol>
</h4>

## 1. Metoda regresji liniowej z scikit-learn 

Tu wykorzystałem pandas do wczytania danych z CSV (większy plik).<br>
Następnie standardowo <b>sklearn.linear_model.LinearRegression</b> oraz <b>sklearn.metrics.mean_squared_error</b>


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
try:
    algen_data = pd.read_csv("algen_data.csv", names = ["target", "a", "b"])
    print("Dane zostały poprawnie wczytane. \nShape algen_data to: %s" % str(algen_data.shape))
except:
    print("Nie udało się wczytać danych, sprawdź lolalizację pliku z danymi")

In [ ]:
algen_data.head(5)

In [ ]:
X = algen_data[["a", "b"]]
X[:5]

In [ ]:
y = algen_data[['target']]
y[:5]

In [ ]:
# zmienna a
np.array(X)[:, :1]

In [ ]:
# zmienna b
np.array(X)[:, 1:]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="darkgrid")

fig, ax = plt.subplots(1, 3, figsize=[16, 6])

ax[0].hist(X.values[:, :1], bins=20, color="cornflowerblue")
ax[0].set_ylim(0, 300)
ax[0].set_xlim(0, 900)
ax[0].set_title("Histogram of Variable a", fontsize="15")
ax[0].grid(True)

ax[1].hist(X.values[:, 1:], bins=20, color="cornflowerblue")
ax[1].set_ylim(0, 300)
ax[1].set_xlim(0, 10500)
ax[1].set_title("Histogram of Variable b", fontsize="15")
ax[1].grid(True)

ax[2].hist(y.values, bins=20, color="brown")
ax[2].set_ylim(0, 300)
ax[2].set_xlim(0, 3500)
ax[2].set_title("Histogram of Target variable (y)", fontsize="15")
ax[2].grid(True)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(1, 2, figsize=[16, 6])

ax[0].plot(X.values[:, :1], y)
ax[0].set_ylabel("Target", fontsize="13")
ax[0].set_xlabel("Variable a", fontsize="13")
ax[0].set_title("Target vs Variable a", fontsize="15")
ax[0].grid(True)

ax[1].plot(X.values[:, 1:], y)
ax[1].set_ylabel("Target", fontsize="13")
ax[1].set_xlabel("Variable b", fontsize="13")
ax[1].set_title("Target vs Variable b", fontsize="15")
ax[1].grid(True)
plt.show()

Jak widać zmienna targetowa (y) jest wysyko skorelowana zarówno ze zmienną a jak i b:

In [ ]:
# Wysoka korelacja między zmiennymi niezależnymi a i b
pearsonr(X.values[:, :1], X.values[:, 1:])

Niestety, zmienne a i b są także wysoko skorelowane między sobą:

In [ ]:
# Oraz między nimi i zmienną targetową
pearsonr(X.values[:, :1], y)

Na razie to zignoruję, chcę zobaczyc jak regresja sobie z tym poradzi:

In [ ]:
lm = LinearRegression().fit(X, y)

print("Regresja liniowa wyestymowała następujące wagi: \nWa = %s, \nWb = %s, \nIntercept = %s"
      % (str(lm.coef_[0][0]), str(lm.coef_[0][1]), str(lm.intercept_[0])))

In [ ]:
print("R square = %.16f" % lm.score(X, y))

In [ ]:
y_prediction = lm.predict(X)
MSE = mean_squared_error(y_true = y, y_pred = y_prediction)
RMSE = sqrt(MSE)

print("MSE = %s, RMSE = %s" % (MSE, RMSE))

### Rozkład reszt i wykresy błędów w zależności od wartości przewidywanej:

In [ ]:
errors = (y_prediction - y).values
SE = errors**2
SE.shape

In [ ]:
mu = errors.mean()
sigma = errors.std()

fig, ax = plt.subplots(1, 2, figsize=[16, 6])

n, bins, patches = ax[0].hist(errors, bins=35, density=True, color="darkseagreen")
ax[0].plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) *  np.exp( - (bins - mu)**2 / (2 * sigma**2) ), linewidth=2, color='r')
ax[0].set_xlim(-0.50, 0.50)
ax[0].set_ylim(0, 4.0)
ax[0].set_title(r"Distribution of residuals $N(\mu=%.3f, \sigma=%.3f)$" % (mu, sigma), fontsize="15")
ax[0].set_xlabel("Residuals", fontsize="13")
ax[0].grid(True)

ax[1].plot(y_prediction, errors, color="darkseagreen")
ax[1].set_xlim(0, 3500)
ax[1].set_xlabel("Predicted values", fontsize="13")
ax[1].set_ylabel("Residuals", fontsize="13")
ax[1].set_title("Residuals by predicted values (+/- 3 $\sigma$)", fontsize="15")
ax[1].axhline(y=errors.std()*3, linewidth=1, color='r', linestyle="--")
ax[1].axhline(y=-errors.std()*3, linewidth=1, color='r', linestyle="--")
ax[1].grid(True)
plt.show()

Reszty, zdają się mieć rozkład normalny ze średnią i dominantą w okolicy zera. Nie widać żadnych zależności dla wartości reszt i predykcji (w całym zakresie predykcji reszty wyglądają na przypadkowe). Dobre dopasowanie modelu potwierdza także bardzo niska i nieistotna korelacja reszt z wartościami przewidywanymi: 

In [ ]:
pearsonr(y_prediction, SE)

## 2. Metoda z użyciem pseudoinwersji macierzy Moore'a-Penrose'a 

Na podstawie podstawowych operacji na macierzach / algebra liniowa:<br>
Poniewarz macierz X ze zmiennymi niezależnymi nie jest prawidłową macierzą z punktu widzenia matematyki, nie będzie miała inwersji i nie można użyć standardowej metody do rozwiązania układu równań liniowych o postaci:
$\vec{y} = A^{-1} \cdot \vec{b}$. Ponieważ pseudoinwersja macierzy Moore'a-Penrose'a spełnia warunek $A \cdot A^+ \approx I$, to macierz $A^+$ będzie maksymalnie zbliżona do $A^{-1}$ co pozwoli obliczyć estymowany wektor współczynników kierunkowych regresji $\hat{y} $ i będzie minimalizowała błąd. Zatem:

$$\hat{y} = A^{+} \cdot \vec{b}$$

Należy pamiętać, aby <b>dodać kolejną kolumnę z 1</b> - to będzie potrzebne aby obliczyć współczynnik dla Intercept, gdyż oryginalnie w macierzy A są tylko kolumny dla zmiennych a i b podczas gdy pełne równanie to:

$$Target = Wa \cdot zmienna\_a + Wb \cdot zmienna\_b + 1 \cdot Intercept$$

I właśnie dla tego $1 \cdot Intercept$ dodaję kolumnę jedynek.
<br>Dla ułatwienia korzystam z np.ones() i pd.assign(nowa_zmienna = ....) a następnie konwertuję X do array.

In [ ]:
A = np.array(X.assign(intercept = np.ones(X.shape[0])))
print(A[:5])
print("Shape: %s" % str(A.shape))

In [ ]:
A_plus = np.linalg.pinv(A)
#print("Macierz pseudoinwersji po przekształceniu: \n %s" % str(A_plus))
A_plus.shape

<br>
Obliczenie współczynników w Python<br>
macierz pseudoinwersji otrzymujemy za pomocą np.linalg.piv(A)
Następnie obliczamy estymatory jako iloczyn macierzy A_plus i wektora ze zmienną target (tu oznaczenie b): $\hat{y} = A^+ \cdot \vec{b}$

In [ ]:
estimator_values = np.linalg.pinv(A).dot(y)
estimator_values

In [ ]:
print("Oszacowania przy pomocy macierzy pseudoinwersji Moore'a i Penrose'a:\nWa = %s, \nWb = %s, \nIntercept: %s"
      % (estimator_values[0], estimator_values[1], estimator_values[2]))
print("\nPonieważ współczynniki są te same co dała regresja liniowa, miary błędów również będą takie same:")
print("MSE = %s, RMSE = %s" % (MSE, RMSE))

## Oraz najlepszy wynik dla algorytmu Algen:

<span style="color: green;">
[*] Najlepsze dopasowanie to: WA = 0.0778, WB = 0.3269, intercept = 24.9667, mse = 0.0140, residual standard error = 0.1184<br>
</span>
Czyli algen całkiem nieźle "odgadł" prawdziwe współczynniki stosując ewolucyjne podejście i losowe mutacje najlepszych rozwiązań


### Bonus: Singular Value Decomposition (SVD)

Jako, że nie jest to macież kwadratowa, to nie można policzyć eigenvectors i eigenvalues jak przy macierzy korelacji na przykład. Dlatego SVD, co także daje podobne efekty: left singular matrix, singular value vector, right singular matrix:

$$A = U \cdot diag(d) \cdot V^{-1}$$

In [ ]:
# UWAGA!
# W pythonie macierz V jest już odwrócona, więc przy obliczaniu spowrotem A nie trzeba jej odwracać! 
U, d, V = np.linalg.svd(A)

In [ ]:
A

In [ ]:
# Rekonstrukcja oryginalnej macierzy za pomocą wszystkich "3 głównych składowych"
U[:, :3].dot(np.diag(d[:3])).dot(V[:3, :])

In [ ]:
# Rekonstrukcja oryginalnej macierzy za pomocą tylko "2 pierwszych głównych składowych"
U[:, :2].dot(np.diag(d[:2])).dot(V[:2, :])

<br>
<br>
W sumie SVD jest wykorzystywane do obliczania nie tylko głównych składowych ale i do macierzy Moore'a Penrose'a (pseudo inwersja macierzy A):

$$A^+ = V \cdot D^+ \cdot U^T$$


Inny sposób, bez SVD ale <b>mniej dokładny</b> to:

$$
A^+ = (A^T \cdot A)^{-1} \cdot A^T
$$

<br>Mniej dokłady sposób bez SVD wtym przypadku daje jednak takie same rezultaty:

In [ ]:
# W A mamy już columnę z 1
A_plus2 = np.linalg.inv(A.T.dot(A)).dot(A.T)

# Wynik (wektor)
A_plus2.dot(y)

In [ ]:
np.trace(A)

In [ ]:
np.linalg.det(A[:A.shape[1], :])

In [ ]:
A.shape